In [1]:
import boto3
import os
import re
from tqdm import tqdm
import sys

In [2]:
os.getcwd()

'/home/jovyan/work/notebooks'

In [3]:
os.chdir('../')

In [4]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf


In [5]:
spark = SparkSession.builder \
        .appName('assignment_1') \
        .getOrCreate() 

In [6]:
df = spark.read.parquet('data/combined_cleaned_data.parquet')

In [7]:
df = df.where("YEAR(pickup_date) <2019 AND YEAR(pickup_date) >2016")

In [8]:
import pyspark.sql.functions as F

In [9]:
df = df.withColumn('trip_duration',F.col("dropoff_date").cast("long") - F.col('pickup_date').cast("long"))

In [10]:
df = df.where("trip_duration >=0")

In [11]:
from pyspark.sql.functions import year, month, dayofmonth


In [12]:
df.select(year("pickup_date")).describe().show()

+-------+------------------+
|summary| year(pickup_date)|
+-------+------------------+
|  count|         236844730|
|   mean|2017.4712318530371|
| stddev|0.4991717049745541|
|    min|              2017|
|    max|              2018|
+-------+------------------+



In [14]:
df.select("trip_duration").describe().show()

+-------+------------------+
|summary|     trip_duration|
+-------+------------------+
|  count|         236844730|
|   mean|1041.9553721672423|
| stddev| 4943.608957631772|
|    min|                 0|
|    max|          45466304|
+-------+------------------+



In [11]:
from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splits=[float('-Inf'),0,300,600,1200,1800,float('Inf') ],inputCol="trip_duration", outputCol="trip_duration_bins")
df = bucketizer.setHandleInvalid("keep").transform(df)

In [12]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

t = {0:"negative time - Invalid",1.0:"0-5 mins", 2.0: "5-10 mins", 3.0:"10-20 mins", 4.0: "20-30 mins", 5.0: 'at least 30 mins'}
convert_to_text = udf(lambda x: t[x], StringType())
df=df.withColumn("trip_duration_bins_string", convert_to_text("trip_duration_bins"))

In [13]:
df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- store_and_fwd_flag: boolean (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- taxi_colour: string (nullable = true)
 |-- pickup_date: timestamp (nullable = true)
 |-- dropoff_date: timestamp (nullable = true)
 |-- trip_duration: long (nullable = true)
 |-- trip_duration_bins: double (nullable = true)
 |-- trip_duration_bins_string: string (nullable = true)



In [14]:
cols = ["VendorID",
 "store_and_fwd_flag",
 "RatecodeID",
 "PULocationID",
 "DOLocationID",
 "passenger_count",
 "trip_distance",
 "extra",
 "mta_tax",
 "tip_amount",
 "tolls_amount",
 "improvement_surcharge",
 "total_amount",
 "payment_type",
 "taxi_colour",
 "pickup_date",
 "dropoff_date",
 "trip_duration",
 "trip_duration_bins_string"]

In [15]:
df = df.select(cols)

In [ ]:
df.write.mode('overwrite').parquet('combined_cleaned_data.parquet')

In [19]:
spark.stop()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38151)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
/usr/local/spark/python/pyspark/context.py:446: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashed, been killed or may also be in a zombie state.
  RuntimeWarning
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38151)
Traceback (most recent call last):
  File "/usr/local

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:38151)